# Sample commands for MagellanMapper tasks

This notebook demonstrates using MagellanMapper through its command-line interface. You can use the script in various ways:
1. Copy the commands to your own Bash shell script
1. Modify this notebook with your own image paths and run blocks for your desired tasks

**Note**: This notebook is a work-in-progress, migrating commands from the [sample commands script](https://github.com/sanderslab/magellanmapper/blob/master/bin/sample_cmds.sh) to here. Please check back for updates!

## Prereqs

- We assume that you've [installed MagellanMapper](https://github.com/sanderslab/magellanmapper#installation) in a Python environment or from the standalone installer
- If you're using a Python environment, activate it (eg `conda activate mag` or `source <path-to-venv>/bin/activate`) before running this Notebook
- Paths are relative to the `magellanmapper` folder
- Running this Notebook requires [JupyterLab](https://jupyter.org/install) and the [Bash kernel](https://github.com/takluyver/bash_kernel) for Jupyter notebooks, which can be installed by:

- Then run JupyterLab:

## Set up image paths

First, let's set up variables to your image paths. The key variable to update is the path to your original image, `img_to_import`. This path minus its extension becomes the "base path" that can be used for most commands, including those on downstream output files.

In [ ]:
# Path to your image file.
img_to_import="/base/path/to/your/image.czi"

# Main image base path. The "base path" is the name of the original image
# file without an extension. MagellanMapper typically outputs files named
# based on it so that you often only need to specify the base path instead of
# exact filenames. For example, after importing img_to_import, the base
# path "/base/path/to/your/image" can be used for most commands.
img="${img_to_import%.*}"

# Downsampled image path. The exact path depends on the type of downsampling
# performed.
img_resized="${img}_resized(456,528,320)"

# Atlas profile for registration tasks. Common profiles are "abaccfv3" for
# the Allen CCFv3 atlas, "abae18pt5" for the Allen Developing Mouse Brain E18.5
# atlas, and "whsrat" for the Waxholm Space rat atlas.
reg_profile=abaccfv3

# set working directory to MagellanMapper folder
if [[ -z "$BASE_DIR" ]]; then
  BASE_DIR="${PWD}/.."
fi
cd "$BASE_DIR"
echo "Set up paths"

## Image import

MagellanMapper typically requires images to be imported into a NumPy format (NPY) for faster access and lower memory usage. We use BioFormats to import from many formats, including proprietary microscopy formats.

### Basic import

To import from a microscopy file, we run the `import_only` processing task. It assumes that the image format includes metadata, but you can also specify metadata as below. The `-v` option is for verbose output, which is not necessary but can help with troubleshooting.

In [ ]:
./run.py --img "$img_to_import" --proc import_only -v

### Import with custom metadata

You can manually specify metadata, which will take precedence over any corresponding settings discovered in the file. `resolutions` are image resolutions in x,y,z order. `magnification` and `zoom` are microscope objective values. See `--set_meta` in the [CLI reference](https://magellanmapper.readthedocs.io/en/latest/cli.html#command-line-argument-reference) for more metadata options.

In [ ]:
./run.py --img "$img_to_import" --proc import_only -v \
  --set_meta resolutions=10.52,10.52,10 magnification=0.63 zoom=1.0

### Import a series of TIF files
<a id="import-tif"></a>

Both single- and multi-plane TIF files can be imported into a volumetric NumPy file.

To import a series of multi-plane TIF files, their filenames should have the format: `name_ch_0.tif`, `name_ch_1.tif`, etc.

In [ ]:
./run.py --img "${img_to_import%.*}.tif" --proc import_only -v

Alternatively, you may have a series of single-plane TIF files. You can put them in a folder and import the whole folder, which will be imported in alphanumerical order. For example, you can import files named: `brain_0000_ch_0.tif`, `brain_0000_ch_1.tif`, `brain_0001_ch_0.tif`, `brain_0001_ch_1.tif`, etc.

In [ ]:
# assumes that img_to_import is a folder containing single-plane TIF files
./run.py --img "$img_to_import" --proc import_only -v

To stitch multi-tile images, use the `pipelines.sh` script instead as described below.

To view the image after import:

In [ ]:
./run.py --img "$img"

### View a TIF file without import

*EXPERIMENTAL*: v1.6 introduced preliminary support for loading TIF files without importing them to NPY format.

Bypass import by including the `.tif` extension when loading the image:

In [ ]:
mm "${img}.tif"

To import the TIF file instead, include the `--proc import_only` argument as [above](#import-tif).

## Atlas Registration

MagellanMapper implements the [Elastix](https://elastix.lumc.nl/) registration toolkit (via [SimpleElastix](https://github.com/SuperElastix/SimpleElastix) to align atlases to your samples. Registering an atlas allows analyses of volume, cell counts, etc by anatomical region.

### Downsample image

Microscopy volumetric images are often large, on the scale of hundreds of GBs to TBs. To make these large files more manageable for image registration, we first downsample images to a smaller volume. MagellanMapper resizes images in blocks to reduce memory requirements.

To rescale an image by a factor, such as a reduction to 25% of each dimension:

In [ ]:
./run.py --img "$img" --proc transform --transform rescale=0.25

To resize to specific dimensions, such as x = 456, y = 528, and z = 320 px:

In [ ]:
./run.py --img "$img" --proc transform --size 456,528,320

It may be desireable to resize the image to the same size at the atlas. The size of the atlas can be stored in atlas profiles, such as our profile for the CCFv3 atlas set above. Specifying this profile will load the size for downsampling:

In [ ]:
./run.py --img "$img" --proc transform --atlas_profile "$reg_profile"

You can also transpose the image to another orientation while downsampling. The original orientation is assumed to be XY, while XZ or YZ are the orthogonal dimensions to it. Here, we rescale and tranpose the image to XZ:

In [ ]:
./run.py --img "$img" --proc transform --transform rescale=0.25 --plane xz

To view the downsampled image, we use the name of output file, which is based on type and amount of downsampling. Here, we use the `img_resized` variable set above, which corresponds to downsampling to the atlas image size:

In [ ]:
./run.py --img "$img_resized"

### Register Atlas to Image

Now that we have downsampled our sample image, we can register an atlas to it. Image registration shifts and morphs a "moving" image to align it to a "fixed" image. To preserve the morphology of the sample image, we make the atlas the "moving" image that moves to fit the sample.

Let's set up the atlas. The registration toolkit supports 2D to 2D or 3D to 3D image registration. Following the conventions for the Allen Insitute atlases, MagellanMapper will look for an atlas intensity image named `atlasVolume.<ext>`, where `ext` can be any extension supported by ITK, such as MHD, NRRD, and NIFTI. The annotated image should be named `annotation.<ext>`.

In [ ]:
# path to atlas directory, assumed to have an atlasVolume and annotation file
atlas_dir="/path/to/your/atlas"

We now register the images by listing the moving image followed by the fixed image.
- `single` registration registers single images as opposed to a group of images
- `--prefix "$img"` outputs image filenames formatted according to the original rather than the resized image

In [ ]:
./run.py --img "$img_resized" "$atlas_dir" --register single --prefix "$img" -v

Atlases are often need reorientation to fit a given sample. Here, we add several example options:
- `--transform rotate=2` rotates the atas by 90 deg x2 (= 180 deg) counter-clockwise
- `--atlas_profile abaccfv3` uses settings in an atlas profile designed for the Allen CCFv3 atlas
- `--channel 1` uses channel 1 (ie the 2nd channel) of the sample for registration

In [ ]:
./run.py --img "$img_resized" "$atlas_dir" --register single -v \
  --transform rotate=2 --atlas_profile abaccfv3 --prefix "$img" --channel 1

To view the registered atlas on the sample, we use the base name of the image and add "registered image suffixes" to specify which registered images to load. Registered images use the base image name and add a suffix depending on the image type, such as `_atlasVolume.mhd` and `_annotation.mhd` for the registered atlas intensity and annotation images, respectively. `_exp.mhd` is the same as the sample image.
- Registered images suffixes are given as: `--reg_suffixes atlas=<suffix> annotation=<suffix> borders=<suffix> fixed_mask=<suffix> moving_mask=<suffix>`, where the type can be omitted if in the given order
- `--roi_profile atlas` can be added to use a grayscale color profile for the intensity image

In [ ]:
./run.py --img "$img" --reg_suffixes exp.mhd annotation.mhd

## Cell Detection

We use a blob detector to locate cells within an image. This detector identifies areas that are bright compared to their surroundings. We use profiles to adjust parameters for blob sizes, sensitivity thresholds, etc.

### Find blobs using the GUI

In the "Detect" panel, pressing the "Detect" button will find all blobs in the current ROI.

### ROI profiles

Profiles consist of parameter settings for tasks such as blob detection. We have included a few default profiles, and you can design custom profiles in YAML files. The "Profiles" panel in the GUI allows selection and viewing of available profiles.

Please see [our docs on profiles](https://magellanmapper.readthedocs.io/en/latest/settings.html#profiles) for more details.

### Find blobs using the CLI

To detect blobs in the whole image, using a profile named "lightsheet":

In [ ]:
./run.py --img "$img" --proc detect --roi_profile lightsheet

To load these detected blobs instead of re-finding blobs in the given ROI:

In [ ]:
./run.py --img "$img" --load blobs

In [ ]:
#### Limit blob detection to an ROI

TODO

### Volume metrics for each atlas region

TODO

### Blob colocalization

TODO

### Build and test ground truth sets

TODO

### Classify cells

The blob detector identifies objects based on contrast to surrounding areas but cannot distinguish between types of objects. A classifier can help to filter out unwanted detections, such as background noise, or identify different cell types.

#### Setup

MagellanMapper uses additional dependencies for the classifier, which can be installed with the `classifier` group:

In [ ]:
pip install -e .[classifier]

#### Train classifier

The classifier trains on labeled images as ground truth. The blob detector can identify objects in your image, which you can label using the annotation tools in the ROI Editor. The classifier learns from these annotations and the images patches around them using a classifier network in Pytorch.

TODO

#### Apply classifier

The trained classifier can be applied to new images through the GUI or CLI.

##### GUI

1. In the "Detect" panel > "Classifier model" field, browse to the trained `.h5` classifier model
1. Check the desired channels in the "Chl" area.
1. Presse the "Detect" button to detect blobs, which will automatically classify them afterward.

Blobs are colored by their class in the ROI Editor. These classes also appear in the "confirmed" column of the blobs table.

##### CLI

To detect and classify blobs, specify the path to your classifier model and run the `detect` task:

In [ ]:
roi_profile="lightsheet"
model_path="/path/to/your/model.h5"
mm "$img" --roi_profile "$roi_profile" --classifier model="$model_path" --proc detect

You can also classify previously detections, which allows repeatedly re-classifying blobs while tweaking the model:

In [ ]:
mm "$img" --classifier model="$model_path" --proc classify --load blobs

Load the blobs to view their classifications in the GUI:

In [ ]:
mm "$img" --load blobs

## 3D Visualization

You can render your images as 3D surfaces with spheres for detected cells. MagellanMapper uses VTK in Mayavi to draw and interact with these surfaces in 3D.

We typically view these 3D images at several levels:
- Downsampled, whole images, since rendering is faster on smaller images
- Cell detections on the full-scale image rescaled to fit the downsampled image
- Individual brain regions from a registered atlas
- ROIs in the full-scale image to view individual microscopic structures

### View full-scale detections on downsampled image

To view cell detections, we use a downsampled image to make rendering times manageable. Cells detected in a full-scale image are scaled to the downsampled image and colored according to a registered atlas. This command has these additional components:
- `--load blob_matches` loads in match-based colocalizations (optional; can be removed)
- `--db <path>` uses a database at a different location, such as one generated on the server where colocalizations were performed (also optional). In this case, we assume that a database file named `magmap.db` is in the image's directory.

In [ ]:
./run.py --img "$img" --roi_profile atlas --reg_suffixes exp.mhd annotation.mhd \
  --load blobs blob_matches -v --db "$(dirname "$img")/magmap.db"

Steps after loading the image to render the image:
- Set the ROI to contain the whole image by setting the offset to x = 0, y = 0, z = 0, and the size to length of each dimension based on the offset sliders' max values
- Leave these default 3D Viewer options checked: raw, surface, no clear
- Change to 3D Viewer (may take a few min to render)
- Overlay blobs: in the Detect panel, select channel and press Detect
- In the Adjust Image panel, reduce the opacity of the main image to see inside the image and start to view blobs inside, which may be very small
- Back in the Detect panel, use the "Scale 3D blobs" slider to adjust blob sizes as desired. Since each change can take a while, it may be faster to enter a value directly into the text box and press Enter.
- Only a fraction of cells are displayed to reduce rendering time. To see a larger proportion of cells, reduce the "3D blobs mask size" slider.

You can also zoom into the 2D images corresponding to a given cell:
- Click on individual blobs to shift the ROI offset to the given blob
- Change to the ROI Editor viewer, reduce the ROI size (eg 50,50,10), and press Redraw

## Heat Maps

Heat or density maps provide another way to visualize whole image cell detections. Cells are grouped into image voxels so that areas of greater cell density are easier to spot as "hot" regions. The resulting image is also a downsampled version of the original, making it more compact to load and display.

### Density map for cell detections

Generating a cell detection density map will load the blobs file from cell detections for the whole image. By default, the density map will have the same size as that of a registered atlas image.

In [ ]:
./run.py -v --img "$img" --register make_density_images

To view the image, we set the `heat.mhd` registered image as the main image. We can also overlay the atlas annotations image and use a profile to show high-contrast colors.

In [ ]:
./run.py --img "$img" --reg_suffixes heat.mhd annotation.mhd --roi_profile contrast

By default, the heat map combines cells from all channels in the image. Use the channel parameter to specify a single or combination of channels. For example, to combine only channels 1 and 2:

In [ ]:
./run.py -v --img "$img" --register make_density_images --channel 1 2

To change the size, provide an alternate shape in `x,y,z` order:

In [ ]:
./run.py -v --img "$img" --register make_density_images --size 200,250,150

## Export Images

TODO

## Image Transformations

TODO

## Generate a new atlas

TODO